# MultiWorkerMirroredStrategy on Hopsworks
---

<font color='red'> <h3>Tested with TensorFlow 2.2.0_rc2</h3></font>

<p>
<h1>Machine Learning on <a href="https://github.com/logicalclocks/hopsworks">Hopsworks
</a></h1> 
</p>

![hops.png](../../../images/hops.png)

## The `hops` python module

`hops` is a helper library for Hops that facilitates development by hiding the complexity of running applications and iteracting with services.

Have a feature request or encountered an issue? Please let us know on <a href="https://github.com/logicalclocks/hops-util-py">github</a>.

### Using the `experiment` module

To be able to run your Machine Learning code in Hopsworks, the code for the whole program needs to be provided and put inside a wrapper function. Everything, from importing libraries to reading data and defining the model and running the program needs to be put inside a wrapper function.

The `experiment` module provides an api to Python programs such as TensorFlow, Keras and PyTorch on a Hopsworks on any number of machines and GPUs.

An Experiment could be a single Python program, which we refer to as an **Experiment**. 

Grid search or genetic hyperparameter optimization such as differential evolution which runs several Experiments in parallel, which we refer to as **Parallel Experiment**. 

ParameterServerStrategy, CollectiveAllReduceStrategy and MultiworkerMirroredStrategy making multi-machine/multi-gpu training as simple as invoking a function for orchestration. This mode is referred to as **Distributed Training**.

### Using the `tensorboard` module
The `tensorboard` module allow us to get the log directory for summaries and checkpoints to be written to the TensorBoard we will see in a bit. The only function that we currently need to call is `tensorboard.logdir()`, which returns the path to the TensorBoard log directory. Furthermore, the content of this directory will be put in as a Dataset in your project's Experiments folder.

The directory could in practice be used to store other data that should be accessible after the experiment is finished.
```python
# Use this module to get the TensorBoard logdir
from hops import tensorboard
tensorboard_logdir = tensorboard.logdir()
```

### Using the `hdfs` module
The `hdfs` module provides a method to get the path in HopsFS where your data is stored, namely by calling `hdfs.project_path()`. The path resolves to the root path for your project, which is the view that you see when you click `Data Sets` in HopsWorks. To point where your actual data resides in the project you to append the full path from there to your Dataset. For example if you create a mnist folder in your Resources Dataset, the path to the mnist data would be `hdfs.project_path() + 'Resources/mnist'`

```python
# Use this module to get the path to your project in HopsFS, then append the path to your Dataset in your project
from hops import hdfs
project_path = hdfs.project_path()
```

```python
# Downloading the mnist dataset to the current working directory
from hops import hdfs
mnist_hdfs_path = hdfs.project_path() + "Resources/mnist"
local_mnist_path = hdfs.copy_to_local(mnist_hdfs_path)
```

### Documentation
See the following links to learn more about running experiments in Hopsworks

- <a href="https://hopsworks.readthedocs.io/en/latest/hopsml/experiment.html">Learn more about experiments</a>
<br>
- <a href="https://hopsworks.readthedocs.io/en/latest/hopsml/hopsML.html">Building End-To-End pipelines</a>
<br>
- Give us a star, create an issue or a feature request on  <a href="https://github.com/logicalclocks/hopsworks">Hopsworks github</a>

### Managing experiments
Experiments service provides a unified view of all the experiments run using the `experiment` module.
<br>
As demonstrated in the gif it provides general information about the experiment and the resulting metric. Experiments can be visualized meanwhile or after training in a TensorBoard.
<br>
<br>
![Image7-Monitor.png](../../../images/experiments.gif)

In [1]:
def mirrored_training():

    import sys
    
    import numpy as np
    import tensorflow as tf
    
    from hops import tensorboard
    from hops import devices
    from hops import hdfs
    
    import pydoop.hdfs as pydoop
    
    data_dir = hdfs.project_path()
    train_filenames = pydoop.path.abspath(data_dir + "TourData/mnist/train/df-mnist_train.tfrecord")
    train_filenames = tf.io.gfile.glob(train_filenames + "/part-r-*")
    validation_filenames = pydoop.path.abspath(data_dir + "TourData/mnist/validation/df-mnist_test.tfrecord")
    validation_filenames = tf.io.gfile.glob(validation_filenames + "/part-r-*")
    
    
    # Define distribution strategy
    strategy = tf.distribute.MirroredStrategy()

    epochs=30 
    steps_per_epoch=5
    validation_steps=2          
    
    batch_size_per_replica = 32
    batch_size = batch_size_per_replica * strategy.num_replicas_in_sync
    shuffle_size = batch_size * 4
    
    num_classes = 10
    kernel = 3
    pool = 2
    dropout = 0.5    

    
    # Input image dimensions
    img_rows, img_cols = 28, 28
    input_shape = (img_rows, img_cols, 1)

    def input_fn(filenames, batch_size):
      
    
      def _parser(serialized_example):
            """Parses a single tf.Example into image and label tensors."""
            features = tf.io.parse_single_example(
                serialized_example,
                features={
                    'image_raw': tf.io.FixedLenFeature([img_rows * img_cols], tf.float32),                    
                    'label': tf.io.FixedLenFeature([], tf.int64),
                })
            
            image = features['image_raw']
            label = features['label']   
                
            return image, label
      
    
      def _normalize_img(image, label):
            """Normalizes images"""
            image = tf.cast(image, tf.float32) / 255
            label = tf.cast(label, tf.int32)        
            return image, label

      def _reshape_img(image, label):
        image = tf.reshape(image, [28, 28, 1])
        # label = tf.one_hot(label, num_classes)
        return image, label
        
        
      # Import MNIST data
      dataset = tf.data.TFRecordDataset(filenames)
        
      # Map the parser over dataset, and batch results by up to batch_size
      dataset = dataset.map(_parser, num_parallel_calls=tf.data.experimental.AUTOTUNE)
        
#      dataset = dataset.map(
#        _normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)

        
      dataset = dataset.map(
        _reshape_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
        
      dataset = dataset.repeat(epochs * steps_per_epoch)
      dataset = dataset.cache()
      dataset = dataset.shuffle(shuffle_size)
      dataset = dataset.batch(batch_size)   
  
      dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
  #    dataset = strategy.experimental_distribute_dataset(dataset)         

      return dataset
    
    model_dir = tensorboard.logdir()
    print('Using %s to store checkpoints.' % model_dir)

    with strategy.scope():
        # Model building/compiling need to be within `strategy.scope()`.
        # Define a Keras Model.    
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.Conv2D(32, kernel_size=(kernel, kernel), padding='same',
                            activation='relu',
                             input_shape=input_shape))
        model.add(tf.keras.layers.Conv2D(64, (kernel, kernel),  padding='same',activation='relu'))
        model.add(tf.keras.layers.MaxPooling2D(pool_size=(pool, pool)))
        model.add(tf.keras.layers.Dropout(dropout))
        model.add(tf.keras.layers.Flatten())
        model.add(tf.keras.layers.Dense(1024, activation='relu'))        
        model.add(tf.keras.layers.Dropout(dropout))
        model.add(tf.keras.layers.Dense(num_classes)) #, activation='softmax'

        model.compile(
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            optimizer=tf.keras.optimizers.Adam(0.001),
            metrics=['accuracy'],
        )
        
    callbacks = [
        tf.keras.callbacks.TensorBoard(log_dir=model_dir, histogram_freq = 1, profile_batch = '500,520'),
        tf.keras.callbacks.ModelCheckpoint(filepath=model_dir),
    ]
    
    model.fit(input_fn(train_filenames, BATCH_SIZE), 
        verbose=0, 
        epochs=epochs, 
        steps_per_epoch=steps_per_epoch,
        validation_data=input_fn(validation_filenames, BATCH_SIZE),
        validation_steps=validation_steps,      
        callbacks=callbacks
    )
    
    score = model.evaluate(input_fn(validation_filenames, BATCH_SIZE), steps=1)
    return {'accuracy': score[1]}    

from hops import experiment
experiment.mirrored(mirrored_training)

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log
15,application_1586507110993_0016,pyspark,idle,Link,Link


SparkSession available as 'spark'.


Could not create experiment (url: /hopsworks-api/api/project/119/experiments/application_1586507110993_0016_1?xattr=REPLACE), server response: 
 HTTP code: 401, HTTP reason: Unauthorized, error code: 200003, error msg: The Token has expired on Sat Apr 11 22:35:15 UTC 2020., user msg: 
Traceback (most recent call last):
  File "/srv/hops/anaconda/anaconda/envs/demo_deep_learning_admin000/lib/python3.6/site-packages/hops/experiment.py", line 594, in mirrored
    experiment_utils._finalize_experiment(experiment_json, None, app_id, run_id, 'FINISHED', duration, logdir, None, None)
  File "/srv/hops/anaconda/anaconda/envs/demo_deep_learning_admin000/lib/python3.6/site-packages/hops/experiment_impl/util/experiment_utils.py", line 689, in _finalize_experiment
    _attach_experiment_xattr(app_id, run_id, experiment_json, 'REPLACE')
  File "/srv/hops/anaconda/anaconda/envs/demo_deep_learning_admin000/lib/python3.6/site-packages/hops/experiment_impl/util/experiment_utils.py", line 461, in _attac